In [ ]:
# pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org seaborn

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup as soup
import os


In [ ]:
csv='../OB/Trailer_Fill.csv'
trailer_fill_df= pd.read_csv(csv)
trailer_fill_df.head()

In [ ]:
trailer_fill_df.dropna(subset=['STORE'], inplace=True)
trailer_fill_df.tail()

In [ ]:
trailer_fill_df['STORE'] = trailer_fill_df['STORE'].astype('int')
trailer_fill_df['DC'] = trailer_fill_df['DC'].astype('int')
# trailer_fill_df['LOAD ID'] = trailer_fill_df['LOAD ID'].astype('int')
# trailer_fill_df['TRAILER ID'] = trailer_fill_df['TRAILER ID'].astype('int')

In [ ]:
trailer_fill_df = trailer_fill_df.drop(trailer_fill_df[trailer_fill_df['STORE'] == 942].index)

In [ ]:
trailer_fill_df.head()

In [ ]:
store_count = len(trailer_fill_df['STORE'].unique())
store_count

In [ ]:
per_store_close = trailer_fill_df.groupby('STORE')['LOAD ID'].count()
per_store_close

In [ ]:
per_store_close = per_store_close.reset_index(name='Close_Count')
per_store_close

In [ ]:
top_25_stores = per_store_close.sort_values("Close_Count", ascending = False)
top_25_stores

In [ ]:
trailer_fill_df.info()

In [ ]:
trailer_fill_df['TRAILER FILL %'] = trailer_fill_df['TRAILER FILL %'].str.rstrip('%').astype(float)/100
trailer_fill_df.head()

In [ ]:
trailer_store_fail = trailer_fill_df[(trailer_fill_df["TRAILER FILL %"] <.65)]
trailer_store_fail.head()

In [ ]:
fail_close = trailer_store_fail.groupby('STORE')['LOAD ID'].count()
fail_close = fail_close.reset_index(name='Failed_Threshold')
fail_close

In [ ]:
complete_trailer_analysis = pd.merge(fail_close,per_store_close, on = 'STORE', how = 'outer')
complete_trailer_analysis

In [ ]:
complete_trailer_analysis['Percent_Fail'] = complete_trailer_analysis['Failed_Threshold'] / complete_trailer_analysis['Close_Count']
complete_trailer_analysis

In [ ]:
complete_trailer_analysis = complete_trailer_analysis.sort_values("Failed_Threshold", ascending = False)
complete_trailer_analysis

In [ ]:
complete_trailer_analysis['Failed_Threshold'] = complete_trailer_analysis['Failed_Threshold'].fillna(0)
complete_trailer_analysis['Percent_Fail'] = complete_trailer_analysis['Percent_Fail'].fillna(0)

In [ ]:
complete_trailer_analysis.head()

In [ ]:
complete_trailer_analysis.to_csv('../OB/trailer_fill_analysis/trailer_fill_analysis.csv', index=True)

In [ ]:
trailer_fill_df.head()

In [ ]:
trailer_fill_df[["Date_Close","Time_Close"]] = trailer_fill_df["TRAILER CLOSE TS (LOCAL)"].str.split(expand=True)
trailer_fill_df.head()

In [ ]:
trailer_fill_df['TRAILER CLOSE TS (LOCAL)'] = pd.to_datetime(trailer_fill_df['TRAILER CLOSE TS (LOCAL)'])
trailer_fill_df.head()

In [ ]:
trailer_fill_df['Weekday']= trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.weekday
trailer_fill_df.head()

In [ ]:
shift_a1_start = '06:00:00'
shift_a1_end = '18:00:00'
shift_a2_start = '18:00:00'
shift_a2_end = '06:00:00'
shift_b1_start = '06:00:00'
shift_b1_end = '16:00:00'
shift_b2_start = '16:00:00'
shift_b2_end = '02:00:00'


trailer_fill_df['A1'] = ((trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek >= 5) | (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek == 0)) & \
                 ((trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time >= pd.to_datetime(shift_a1_start).time()) & \
                  (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time <= pd.to_datetime(shift_a1_end).time()))

trailer_fill_df['A2'] = ((trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek >= 5) | (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek <= 1)) & \
                 ((trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time >= pd.to_datetime(shift_a2_start).time()) | \
                  (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time <= pd.to_datetime(shift_a2_end).time()))

trailer_fill_df['B1'] = (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek >= 1) & (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek <= 4) & \
                 ((trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time >= pd.to_datetime(shift_b1_start).time()) & \
                  (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time <= pd.to_datetime(shift_b1_end).time()))

trailer_fill_df['B2'] = (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek >= 1) & (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.dayofweek <= 5) & \
                 ((trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time >= pd.to_datetime(shift_b2_start).time()) | \
                  (trailer_fill_df['TRAILER CLOSE TS (LOCAL)'].dt.time <= pd.to_datetime(shift_b2_end).time()))

trailer_fill_df.head()

In [ ]:
trailer_fill_df['Shift']= np.where(trailer_fill_df['A1'], 'A1',
                                np.where(trailer_fill_df['A2'], 'A2',
                                np.where(trailer_fill_df['B1'], 'B1',
                                np.where(trailer_fill_df['B2'], 'B2',
                                'B3'))))

trailer_fill_df.head()

In [ ]:
trailer_fail_shift = trailer_fill_df[(trailer_fill_df["TRAILER FILL %"] <.65)]
trailer_fail_shift.head()

In [ ]:
total_trailer_key = trailer_fill_df.groupby('Shift')['LOAD ID'].count()
total_trailer_key = total_trailer_key.reset_index(name='Total_Trailer')
total_trailer_key

In [ ]:
trailer_fail_shift = trailer_fail_shift.groupby('Shift')['LOAD ID'].count()
trailer_fail_shift = trailer_fail_shift.reset_index(name='Failed_Threshold')
trailer_fail_shift.head()

In [ ]:
total_trailer = pd.merge(trailer_fail_shift, total_trailer_key, how='left', on=['Shift', 'Shift'])
total_trailer

In [ ]:
total_trailer.to_csv('../OB/trailer_fill_analysis/trailer_fill_shift.csv', index=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
import matplotlib as mpl

In [ ]:
trailer_fail_plot = trailer_fill_df[(trailer_fill_df["TRAILER FILL %"] <.65)]
trailer_fail_plot.head()

In [ ]:
trailer_fail_plot['TRAILER CLOSE TS (LOCAL)'] = trailer_fail_plot['TRAILER CLOSE TS (LOCAL)'].dt.hour
trailer_fail_plot.head()

In [ ]:
heatmap_data = trailer_fail_plot.pivot_table(index='TRAILER CLOSE TS (LOCAL)', columns='Weekday', values='LOAD ID', aggfunc='count')

plt.figure(figsize=(5, 4))
plt.imshow(heatmap_data, cmap='YlOrRd', aspect='auto', origin='lower')
plt.colorbar(label='Count of Misses')
plt.xticks(np.arange(len(heatmap_data.columns)), heatmap_data.columns, rotation=45)
plt.yticks(np.arange(len(heatmap_data.index)), heatmap_data.index)
plt.xlabel('Weekday (0 is Monday & 6 is Sunday)')
plt.ylabel('Hour')
plt.title('Missed Threshold by Hour')
plt.savefig('../OB/trailer_fill_analysis/trailer_fill_hour.png')
plt.show()